<a href="https://colab.research.google.com/github/papipopapu/PhysicsThings/blob/main/FluidSym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import numba as nb
import typing as tp

In [2]:
from scipy.interpolate import RectBivariateSpline 

In [79]:



class fluidThings:
  def __init__(self, Ni: int, Nj: int, dh: float):
    self.__Ni = Ni # Number of fluid cells on a dimension
    self.__Nj = Nj
    self.__dh = dh # Size of a cell 

    self.__u = np.arange(Ni*(Nj+1)).reshape(Ni, Nj+1) # Field of vx velocities in walls (named u), to zeros when we can initialize initial conditions with method
    self.__uSpline = RectBivariateSpline(np.arange(Ni), np.arange(Nj+1), self.__u) # Smooth interpolation
   # print (self.__u ) # to remove check
 
    self.__v = np.arange(Nj*(Ni+1)).reshape(Ni+1, Nj) # Field of vy velocities in walls (named v)
    self.__vSpline = RectBivariateSpline(np.arange(Ni+1), np.arange(Nj), self.__v)
   # print (self.__v )

    self.__p = np.arange((Ni+2)*(Nj+2)).reshape(Ni+2, Nj+2)
    self.__pSpline = RectBivariateSpline(np.arange(Ni+2), np.arange(Nj+2), self.__p)  # presssure, is defined on solid cells -> n+2
    print (self.__p )

    self.__ink = np.arange((Ni+2)*(Nj+2)).reshape(Ni+2, Nj+2)
    self.__inkSpline = RectBivariateSpline(np.arange(Ni+2), np.arange(Nj+2), self.__p) # zero on solid cell walls?
   # print (self.__ink ) # to remove check

  @staticmethod
  @nb.njit()
  def trimmedDx(Field: np.array): # its trimmed specially to be added to velocities in x
    trim = Field[1:-1, :]
    Dx = trim[:, 1:] - trim[:, :-1]
    return Dx

  @staticmethod
  @nb.njit()
  def trimmedDy(Field: np.array): # its trimmed specially to be added to velocities in x
    trim = Field[:, 1:-1]
    Dy = trim[1:, :] - trim[:-1, :]
    return Dy

  @staticmethod
  @nb.njit()
  def uIdx(r: np.array, Ni: int, dh: float): # idx = [[ui],[uj],[vi],[vj]], this functions gets x,y and returns indices for u and v in np.array
    idx = np.zeros((4, r[0].size));
    xScaled = r[0]/dh 
    yScaled = r[1]/dh

    idx[0] = -yScaled + Ni - 0.5 # u_i
    idx[1] = xScaled # u_j
    idx[2] = -yScaled + Ni # v_i
    idx[3] = xScaled - 0.5 # v_j

    return idx
  
  @staticmethod
  @nb.njit()
  def sIdx(r: np.array, Ni: int, dh: float): # idx = [[si],[sj]] , for ink or pressure
    idx = np.zeros((2, r[0].size));

    idx[0] = Ni - r[1]/dh + 0.5 # u_j
    idx[1] = r[0]/dh + 0.5
    return idx


  def fu(self, r: np.array): # r = [[x],[y]], return [[u],[v]]

    idx = self.uIdx(r, self.__Ni, self.__dh); # get indices
    uVector = np.vstack((self.__uSpline.ev(idx[0], idx[1]), self.__vSpline.ev(idx[2], idx[3]))) # interpolate whole vector

    return uVector

  def fp(self, r: np.array): # return [p]
    idx = self.sIdx(r, self.__Ni, self.__dh); # get indices
    p = self.__pSpline.ev(idx[0], idx[1])

    return p


  def fink(self, r: np.array): # return [p]
    idx = self.sIdx(r, self.__Ni, self.__dh); # get indices
    ink = self.__inkSpline.ev(idx[0], idx[1])

    return ink


  def test(self): # for testing
    f = self.trimmedDx(self.__p)
    print(f)
    return f - self.__u
    



In [83]:
fluid = fluidThings(1000, 1000, 1)

[[      0       1       2 ...     999    1000    1001]
 [   1002    1003    1004 ...    2001    2002    2003]
 [   2004    2005    2006 ...    3003    3004    3005]
 ...
 [1000998 1000999 1001000 ... 1001997 1001998 1001999]
 [1002000 1002001 1002002 ... 1002999 1003000 1003001]
 [1003002 1003003 1003004 ... 1004001 1004002 1004003]]


In [85]:
r = np.array([[0],[0.5]]) 
u = fluid.fu(np.random.rand(2, 10000)*10000)
u

array([[  1000.        , 262926.29567523,   1000.        , ...,
          1000.        ,   1000.        ,   1000.        ],
       [   999.        , 263163.63104419,    999.        , ...,
           999.        ,    999.        ,    999.        ]])

In [82]:
fluid.test().shape

[[1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1]]


(10, 11)

In [6]:
x = np.array([1,2,3])
x[:-1]

array([1, 2])

In [20]:
fluid.fu(r)

array([[1001.03319249, 1003.95290786, 1000.04074744, ..., 1001.50902772,
        1004.4828632 , 1003.13251249],
       [ 909.53319249,  912.45290786,  909.        , ...,  910.00902772,
         912.9828632 ,  911.63251249]])

In [27]:
fluid.fink(r)

array([1016.94805678, 1010.92050534, 1014.4855244 , 1015.22773182,
       1015.90271581, 1016.36526719, 1017.36502357, 1011.51787846,
       1014.15835773, 1013.54370803, 1010.53980865, 1014.56789699,
       1017.73438612, 1012.23431874, 1011.76043485, 1018.12036436,
       1017.86678512, 1016.31974749, 1015.86212223, 1017.16207977,
       1010.84740851, 1014.68153721, 1012.03083438, 1012.29248931,
       1014.05651594, 1016.9892884 , 1013.34304161, 1012.2680132 ,
       1010.76159665, 1012.53279701, 1012.80612306, 1017.45438881,
       1015.88338282, 1017.59456725, 1015.31134701, 1014.02465632,
       1013.87050777, 1013.69041458, 1015.56040445, 1013.97798919,
       1018.27834495, 1017.06707598, 1015.98196396, 1013.82839575,
       1016.4973971 , 1015.54597365, 1017.17558828, 1010.9849063 ,
       1013.45317444, 1015.28189115, 1016.00309061, 1014.73831942,
       1012.97841819, 1010.71354335, 1013.19219144, 1012.31204576,
       1012.19939764, 1017.76082958, 1013.79076884, 1012.95979